In [36]:
# Import context
import sys
import os
sys.path.insert(0, os.path.abspath('..\\matlib'))
sys.path.append('..')  # Add parent directory (root)
print(sys.path)

from vec import Vec
# from mat import Mat

# Check Python version (critical for solver.py)
import imp
print(imp.get_magic())
print(imp.get_magic()==b'\xee\x0c\r\n')  # should be true

['c:\\code\\sandbox\\anaconda\\Coding the matrix\\matlib', 'c:\\code\\sandbox\\anaconda\\Coding the matrix\\matlib', 'c:\\code\\sandbox\\anaconda\\Coding the matrix\\matlib', 'c:\\code\\sandbox\\anaconda\\Coding the matrix\\matlib', '', 'C:\\Anaconda3\\envs\\py34\\python34.zip', 'C:\\Anaconda3\\envs\\py34\\DLLs', 'C:\\Anaconda3\\envs\\py34\\lib', 'C:\\Anaconda3\\envs\\py34', 'c:\\anaconda3\\envs\\py34\\lib\\site-packages\\setuptools-20.7.0-py3.4.egg', 'C:\\Anaconda3\\envs\\py34\\lib\\site-packages', 'C:\\Anaconda3\\envs\\py34\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\Spiridonov\\.ipython', '..', '..', '..', '..']
b'\xee\x0c\r\n'
True


In [4]:
# nested comprehension for list-of-row-list representation
# of 3x4 matrix all of whose elements are zero
[[0 for _ in range(4)] for _ in range(3)]

[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]

In [33]:
class Mat:
    def __init__(self, labels, function):
        self.D = labels
        self.f = function

# Identity matrix
def identity(D): return Mat((D,D), {(k,k):1 for k in D})

"""
matrix = [ [1, 2, 3], [10, 20, 30] ]

row-labels -> matrix.D[0]: 'a', 'b'
col-labels -> matrix.D[1]: '@', '#', '?'
"""

matrix = Mat( ({'a','b'}, {'@', '#', '?'}), {('a','@'):1, ('a','#'):2, ('a','?'):3, ('b','@'):10, ('b','#'):20, ('b','?'):30} )

# row to column dictionary conversion
def mat2coldict(A):
    return {c: Vec(A.D[0], {r:A.f[r,c] for r in A.D[0]}) for c in A.D[1]}

mat2coldict(matrix)

{'#': Vec({'b', 'a'},{'b': 20, 'a': 2}),
 '?': Vec({'b', 'a'},{'b': 30, 'a': 3}),
 '@': Vec({'b', 'a'},{'b': 10, 'a': 1})}

In [31]:
!python -m doctest mat.py
# !python -m doctest mat_sparsity.py

**********************************************************************
File "c:\code\sandbox\anaconda\Coding the matrix\l03_matrix\mat.py", line 65, in mat.Mat.__eq__
Failed example:
    C == A
Exception raised:
    Traceback (most recent call last):
      File "C:\Anaconda3\lib\doctest.py", line 1320, in __run
        compileflags, 1), test.globs)
      File "<doctest mat.Mat.__eq__[7]>", line 1, in <module>
        C == A
      File "c:\code\sandbox\anaconda\Coding the matrix\l03_matrix\mat.py", line 74, in equal
        if A[i] != B[i]:
      File "c:\code\sandbox\anaconda\Coding the matrix\l03_matrix\mat.py", line 36, in getitem
        assert k[0] in M.D[0] and k[1] in M.D[1]
    AssertionError
**********************************************************************
1 items had failures:
   1 of   9 in mat.Mat.__eq__
***Test Failed*** 1 failures.


In [34]:
# Solving Lights Out
from GF2 import zero, one

n = 2
D = {(i,j) for i in range(n) for j in range(n)}

# Button vectors
b00 = Vec(D, {(0,0):one, (0,1):one, (1,0):one})
b01 = Vec(D, {(0,0):one, (0,1):one, (1,1):one})
b10 = Vec(D, {(0,0):one, (1,0):one, (1,1):one})
b11 = Vec(D, {(0,1):one, (1,0):one, (1,1):one})

# Initial position (lights in)
b = Vec(D, {(0,0):one, (1,0):one})
print(b)

from matutil import coldict2mat
A = coldict2mat({(0,0):b00, (0,1):b01, (1,0):b10, (1,1):b11})
print(A)

from solver import solve
x = solve(A, b)
print(x)
A*x == b


 (0, 0) (0, 1) (1, 0) (1, 1)
----------------------------
    one      0    one      0

            (0, 0) (0, 1) (1, 0) (1, 1)
          -----------------------------
 (0, 0)  |     one    one    one      0
 (0, 1)  |     one    one      0    one
 (1, 0)  |     one      0    one    one
 (1, 1)  |       0    one    one    one


 (0, 0) (0, 1) (1, 0) (1, 1)
----------------------------
      0    one      0    one


True

In [37]:
from matutil import listlist2mat
from solver import solve
A = listlist2mat([[1,3],[5,7]])
b = Vec({0,1}, {0:1, 1:1})
u = solve(A, b)
print(u)
b - A*u  # if the residual is small => success


    0   1
---------
 -0.5 0.5


Vec({0, 1},{0: -4.440892098500626e-16, 1: -8.881784197001252e-16})

In [24]:
# Resource consumption (espionage) problem
from mat import Mat
col_labels={'metal','concrete','plastic','water','electricity'}
row_labels={'gnome','hoop','slinky','putty','shooter'}
M = Mat((row_labels,col_labels),
        {('gnome','concrete'):1.3,
         ('gnome','plastic'):.2,
         ('gnome','water'):.8,
         ('gnome','electricity'):.4})
#etc.

# Observed resource utilization
b = Vec(col_labels, {'electricity':1409.5, 'water':1485.0, 
                     'concrete':1300.0, 'metal':226.25,
                     'plastic':677.0})
u = solve(M.transpose(), b)
print(u)
print(u*M)  # check that solution makes sense
res = b-u*M
print(res*res)


    gnome hoop putty shooter slinky
-----------------------------------
 1.41E+03    0     0       0      0

 concrete electricity metal plastic    water
--------------------------------------------
 1.84E+03         566     0     283 1.13E+03
1333583.5733695652


In [17]:
# For some matrix-vector equations, there is no solution
# For these, the res(idual) will not be so small
from vecutil import list2vec
A = listlist2mat([[1,2], [4,5], [-6,1]])
b = list2vec([1,1,1])
u = solve(A, b) 
    # Computes the solution with res as small as possible
res = b-A*u
print(res*res)

0.24287856071964012


In [28]:
# Sensor node problem
D = {'radio', 'sensor', 'memory', 'CPU'}
duration1 = Vec(D, {'radio':.1, 'CPU':.3})
duration2 = Vec(D, {'sensor':.2, 'CPU':.4})
duration3 = Vec(D, {'memory':.3, 'CPU':.1})
duration4 = Vec(D, {'memory':.5, 'CPU':.4})

from matutil import rowdict2mat
A = rowdict2mat({1:duration1, 2:duration2, 3:duration3, 4:duration4})
print(A)
b = Vec({1, 2, 3, 4},{1: 140.0, 2: 170.0, 3: 60.0, 4: 170.0})
rate = solve(A,b)
print(rate)
res = b - A*rate
print(res*res)  # note: not necessarily the only solution


       CPU memory radio sensor
     -------------------------
 1  |  0.3      0   0.1      0
 2  |  0.4      0     0    0.2
 3  |  0.1    0.3     0      0
 4  |  0.4    0.5     0      0


 CPU memory radio sensor
------------------------
 300    100   500    250
1.0097419586828951e-27


In [18]:
# Some matrix-vector equations are ill-conditioned, which
# can prevent an algorithm using floats from getting
# even approx. solutions, even if solution exists!
A = listlist2mat([[1e20,1],[1,0]])
b = list2vec([1,1])
u = solve(A, b)
print(u)
res = b - A*u
print(res*res)


     0 1
--------
 1E-20 0
1.0
